In [ ]:
import torchvision.transforms as T
from torch.utils.data import Subset, Dataset, DataLoader, random_split
from torch import nn
import torch.nn.functional
from torchvision.datasets import ImageFolder
from torchvision.io import read_image, ImageReadMode
import torch
import pickle as pkl


<h1>Trying a predefined model (Resnet)</h1>

In [ ]:

dataset_path = "../informatie/apple_disease_classification/images/Train/Dataset/"

In [ ]:
class DatasetAppels(Dataset):
    def __init__(self, img_folder_path, transform):

        image_folder = ImageFolder(img_folder_path, transform=transform)
        self.images = [image[0] for image in image_folder]
        self.labels = image_folder.targets
        self.class_dict = image_folder.class_to_idx

    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        return [self.images[idx], self.labels[idx]]
    


In [ ]:
preprocess = T.Compose([
    T.Resize(256),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
dataset = DatasetAppels(dataset_path, preprocess)

dataset

In [ ]:
# Resnet variants:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=False)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet34', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet152', pretrained=False)
model.train()